# Test de connexion SQL Server

Ce notebook teste différentes configurations pour se connecter à SQL Server avec pyodbc.

**Erreur commune:** `Named Pipes Provider: Could not open a connection` suggère un problème de:
- Nom du serveur incorrect
- Port ou instance manquante
- Serveur non accessible (firewall, VPN, etc.)
- SQL Server non configuré pour les connexions distantes

In [1]:
import pyodbc
import os
from dotenv import load_dotenv
from pathlib import Path

# Charger les variables d'environnement
load_dotenv()

SERVER = os.getenv('SERVER_NAME')
DATABASE = os.getenv('DATABASE_NAME')
USERNAME = os.getenv('SQL_LOGIN_USER')
PASSWORD = os.getenv('SQL_LOGIN_PASSWORD')

print(f"Configuration:")
print(f"  Server: {SERVER}")
print(f"  Database: {DATABASE}")
print(f"  User: {USERNAME}")
print(f"  Password: {'*' * len(PASSWORD) if PASSWORD else 'NOT SET'}")

Configuration:
  Server: Zeus11
  Database: france_172074
  User: user_breau
  Password: ****************


In [2]:
# Vérifier les drivers ODBC disponibles
print("Drivers ODBC disponibles sur cette machine:")
for driver in pyodbc.drivers():
    print(f"  - {driver}")

Drivers ODBC disponibles sur cette machine:
  - SQL Server
  - ODBC Driver 17 for SQL Server
  - Microsoft Access Driver (*.mdb, *.accdb)
  - Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
  - Microsoft Access Text Driver (*.txt, *.csv)
  - Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)


## Test 1: Configuration basique

In [3]:
# Test avec ODBC Driver 17 + Encrypt=no (le plus compatible)
def test_connection(server_name, description):
    """
    Teste la connexion avec un nom de serveur donné.
    """
    print(f"\n{'='*70}")
    print(f"Test: {description}")
    print(f"Server: {server_name}")
    print(f"{'='*70}")
    
    conn_str = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server_name};"
        f"DATABASE={DATABASE};"
        f"UID={USERNAME};"
        f"PWD={PASSWORD};"
        f"Encrypt=no;"
        f"Connection Timeout=5;"
    )
    
    try:
        print("Tentative de connexion...")
        conn = pyodbc.connect(conn_str)
        
        print("✓ CONNEXION REUSSIE!")
        
        # Tester avec une requête
        cursor = conn.cursor()
        cursor.execute("SELECT @@SERVERNAME AS ServerName, @@VERSION AS Version")
        row = cursor.fetchone()
        
        print(f"\nInformations du serveur:")
        print(f"  Nom: {row.ServerName}")
        print(f"  Version: {row.Version[:100]}...")
        
        # Lister quelques tables
        cursor.execute("""
            SELECT TOP 10 TABLE_SCHEMA, TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
            ORDER BY TABLE_NAME
        """)
        
        print(f"\nExemples de tables dans {DATABASE}:")
        for row in cursor.fetchall():
            print(f"  - {row.TABLE_SCHEMA}.{row.TABLE_NAME}")
        
        cursor.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"✗ ECHEC")
        error_msg = str(e)
        
        if 'login failed' in error_msg.lower():
            print("  Raison: Mauvais credentials (mais serveur trouvé!)")
        elif 'timeout' in error_msg.lower():
            print("  Raison: Timeout de connexion")
        elif 'named pipes' in error_msg.lower():
            print("  Raison: Erreur Named Pipes (serveur non trouvé ou inaccessible)")
        elif 'network' in error_msg.lower():
            print("  Raison: Erreur réseau")
        
        print(f"\nMessage d'erreur complet:")
        print(f"  {error_msg}")
        
        return False

# Test avec le nom de serveur de base
test_connection(SERVER, "Nom de serveur de base (.env)")


Test: Nom de serveur de base (.env)
Server: Zeus11
Tentative de connexion...
✗ ECHEC
  Raison: Timeout de connexion

Message d'erreur complet:
  ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [64].  (64) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (64)')


False

## Test 2: Variations du nom de serveur

Essayons différents formats de noms de serveur:

In [ ]:
# Test avec différentes variations
server_variants = [
    (f"{SERVER},1433", "Port par défaut explicite (,1433)"),
    (f"{SERVER}\\SQLEXPRESS", "Instance SQLEXPRESS"),
    (f"{SERVER}\\MSSQLSERVER", "Instance MSSQLSERVER"),
    (f"tcp:{SERVER}", "Protocole TCP explicite"),
    (f"tcp:{SERVER},1433", "TCP + Port 1433"),
    (f"{SERVER}.local", "Suffixe .local"),
]

for server_name, description in server_variants:
    success = test_connection(server_name, description)
    if success:
        print(f"\n\n*** CONFIGURATION GAGNANTE ***")
        print(f"Utilisez ce format de serveur: {server_name}")
        break

## Test 3: Variations des options de connexion

Si le serveur est trouvé mais la connexion échoue, testons différentes options:

In [ ]:
# Modifier le nom du serveur ci-dessous si nécessaire
SERVER_TO_TEST = SERVER  # Ou mettez le format qui a marché ci-dessus

connection_options = [
    {
        'name': 'Encrypt=no (le plus compatible)',
        'options': 'Encrypt=no;Connection Timeout=5;'
    },
    {
        'name': 'TrustServerCertificate=yes',
        'options': 'TrustServerCertificate=yes;Connection Timeout=5;'
    },
    {
        'name': 'Encrypt=yes + TrustServerCertificate=yes',
        'options': 'Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=5;'
    },
    {
        'name': 'Driver SQL Server (ancien)',
        'driver': 'SQL Server',
        'options': 'Connection Timeout=5;'
    },
]

for config in connection_options:
    driver = config.get('driver', 'ODBC Driver 17 for SQL Server')
    options = config['options']
    
    conn_str = (
        f"DRIVER={{{driver}}};"
        f"SERVER={SERVER_TO_TEST};"
        f"DATABASE={DATABASE};"
        f"UID={USERNAME};"
        f"PWD={PASSWORD};"
        f"{options}"
    )
    
    print(f"\n{'='*70}")
    print(f"Test: {config['name']}")
    print(f"{'='*70}")
    print(f"Connection string: {conn_str.replace(PASSWORD, '***')}")
    
    try:
        conn = pyodbc.connect(conn_str)
        print("✓ SUCCESS!")
        conn.close()
        break
    except Exception as e:
        print(f"✗ FAILED: {str(e)[:150]}")

## Test 4: Connection string finale

Une fois que tu as trouvé la bonne configuration, teste-la ici:

In [ ]:
# CONFIGURATION FINALE - Modifie selon les tests ci-dessus
FINAL_SERVER = SERVER  # Modifie si nécessaire (ex: "Zeus11,1433" ou "Zeus11\\SQLEXPRESS")

connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={FINAL_SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
    f"Encrypt=no;"  # Modifie si nécessaire
)

print("Connection string finale:")
print(connection_string.replace(PASSWORD, '***'))
print()

try:
    conn = pyodbc.connect(connection_string)
    print("✓ CONNEXION ETABLIE!")
    
    cursor = conn.cursor()
    
    # Quelques requêtes de test
    print("\nTest de requêtes:")
    
    cursor.execute("SELECT DB_NAME() AS CurrentDB")
    print(f"  Base de données courante: {cursor.fetchone().CurrentDB}")
    
    cursor.execute("SELECT SUSER_NAME() AS CurrentUser")
    print(f"  Utilisateur courant: {cursor.fetchone().CurrentUser}")
    
    cursor.execute("SELECT @@VERSION AS Version")
    version = cursor.fetchone().Version
    print(f"  Version SQL Server: {version.split(chr(10))[0]}")
    
    cursor.close()
    conn.close()
    
    print("\n" + "="*70)
    print("TOUT FONCTIONNE! Tu peux utiliser cette configuration.")
    print("="*70)
    
except Exception as e:
    print(f"✗ Erreur: {e}")

## Debugging supplémentaire

Si rien ne marche, voici quelques choses à vérifier:

In [ ]:
print("Checklist de debugging:")
print("")
print("1. Le serveur SQL Server est-il démarré?")
print("   - Vérifier avec l'IT ou SQL Server Management Studio")
print("")
print("2. Es-tu sur le bon réseau / VPN?")
print("   - Certains serveurs SQL ne sont accessibles que via VPN")
print("")
print("3. Le serveur SQL Server accepte-t-il les connexions TCP/IP?")
print("   - Vérifier dans SQL Server Configuration Manager")
print("   - TCP/IP doit être activé")
print("")
print("4. Le firewall bloque-t-il le port 1433?")
print("   - Tester avec: telnet Zeus11 1433")
print("   - Ou: Test-NetConnection -ComputerName Zeus11 -Port 1433 (PowerShell)")
print("")
print("5. Le nom du serveur est-il correct?")
print("   - Essayer avec l'adresse IP au lieu du nom")
print("   - Demander à l'IT le nom complet (FQDN) ou l'IP")
print("")
print("6. Y a-t-il une instance nommée?")
print("   - Le serveur pourrait être 'Zeus11\\INSTANCE_NAME'")
print("   - Demander à l'IT le nom de l'instance SQL Server")
print("")
print("7. Les credentials sont-ils corrects?")
print(f"   - User: {USERNAME}")
print("   - Si erreur 'Login failed', les credentials sont incorrects")
print("")

# Test de ping
import subprocess

print("\nTest de ping du serveur:")
try:
    result = subprocess.run(['ping', '-n', '1', SERVER], capture_output=True, text=True, timeout=5)
    if 'TTL=' in result.stdout or 'ttl=' in result.stdout:
        print(f"  ✓ Le serveur {SERVER} est accessible via ping")
    else:
        print(f"  ✗ Le serveur {SERVER} ne répond pas au ping")
        print("    (Cela ne veut pas forcément dire que SQL Server est inaccessible)")
except Exception as e:
    print(f"  ✗ Erreur lors du ping: {e}")